In [167]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import category_encoders as ce

In [142]:
pd.options.display.max_rows=500
pd.options.display.max_columns

In [153]:
train_df = pd.read_csv("TrainingWiDS2021.csv")
#test_df = pd.read_csv("UnlabeledWiDS2021.csv")

In [24]:
train_df.head()

,Unnamed: 0,encounter_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,...,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,diabetes_mellitus
0,1,214826,118,68.0,22.732803,0,Caucasian,M,180.3,Floor,...,NaN,NaN,0,0,0,0,0,0,0,1
1,2,246060,81,77.0,27.421875,0,Caucasian,F,160.0,Floor,...,51.0,51.0,0,0,0,0,0,0,0,1
2,3,276985,118,25.0,31.952749,0,Caucasian,F,172.7,Emergency Department,...,NaN,NaN,0,0,0,0,0,0,0,0
3,4,262220,118,81.0,22.635548,1,Caucasian,F,165.1,Operating Room,...,337.0,337.0,0,0,0,0,0,0,0,0
4,5,201746,33,19.0,NaN,0,Caucasian,M,188.0,NaN,...,NaN,NaN,0,0,0,0,0,0,0,0


In [136]:
train_df = train_df.iloc[:,1:]

In [25]:
print(train_df.info())
print(test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130157 entries, 0 to 130156
Columns: 181 entries, Unnamed: 0 to diabetes_mellitus
dtypes: float64(157), int64(18), object(6)
memory usage: 179.7+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10234 entries, 0 to 10233
Columns: 180 entries, Unnamed: 0 to solid_tumor_with_metastasis
dtypes: float64(156), int64(18), object(6)
memory usage: 14.1+ MB
None


In [26]:
train_df.isnull().sum()

Unnamed: 0                          0
encounter_id                        0
hospital_id                         0
age                              4988
bmi                              4490
elective_surgery                    0
ethnicity                        1587
gender                             66
height                           2077
hospital_admit_source           33198
icu_admit_source                  240
icu_id                              0
icu_stay_type                       0
icu_type                            0
pre_icu_los_days                    0
readmission_status                  0
weight                           3463
albumin_apache                  78163
apache_2_diagnosis               1685
apache_3j_diagnosis               865
apache_post_operative               0
arf_apache                          0
bilirubin_apache                82560
bun_apache                      25411
creatinine_apache               24882
fio2_apache                     99720
gcs_eyes_apa

In [154]:
len(train_df)*.9

117141.3

In [155]:
thresh = len(train_df) * .9
train_df.dropna(thresh = thresh, axis = 1, inplace = True)

In [156]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130157 entries, 0 to 130156
Data columns (total 78 columns):
Unnamed: 0                     130157 non-null int64
encounter_id                   130157 non-null int64
hospital_id                    130157 non-null int64
age                            125169 non-null float64
bmi                            125667 non-null float64
elective_surgery               130157 non-null int64
ethnicity                      128570 non-null object
gender                         130091 non-null object
height                         128080 non-null float64
icu_admit_source               129917 non-null object
icu_id                         130157 non-null int64
icu_stay_type                  130157 non-null object
icu_type                       130157 non-null object
pre_icu_los_days               130157 non-null float64
readmission_status             130157 non-null int64
weight                         126694 non-null float64
apache_2_diagnosis        

In [159]:
encoder = ce.OneHotEncoder(cols = ["ethnicity", "gender", "icu_admit_source",
"icu_stay_type","icu_type","apache_2_diagnosis","apache_3j_diagnosis"], handle_unknown='return_nan', return_df=True, use_cat_names=True)

In [160]:
df_encoded = encoder.fit_transform(train_df)

In [141]:
df_encoded.head()

,ethnicity_Caucasian,ethnicity_nan,ethnicity_Hispanic,ethnicity_African American,ethnicity_Asian,ethnicity_Native American,ethnicity_Other/Unknown,gender_M,gender_F,gender_nan,hospital_admit_source_Floor,hospital_admit_source_Emergency Department,hospital_admit_source_Operating Room,hospital_admit_source_nan,hospital_admit_source_Direct Admit,hospital_admit_source_Other Hospital,hospital_admit_source_Other ICU,hospital_admit_source_ICU to SDU,hospital_admit_source_Recovery Room,hospital_admit_source_Chest Pain Center,hospital_admit_source_Step-Down Unit (SDU),hospital_admit_source_Acute Care/Floor,hospital_admit_source_PACU,hospital_admit_source_Observation,hospital_admit_source_ICU,hospital_admit_source_Other,icu_admit_source_Floor,icu_admit_source_Accident & Emergency,icu_admit_source_Operating Room / Recovery,icu_admit_source_Other Hospital,icu_admit_source_Other ICU,icu_admit_source_nan,icu_stay_type_admit,icu_stay_type_readmit,icu_stay_type_transfer,icu_type_CTICU,icu_type_Med-Surg ICU,icu_type_CCU-CTICU,icu_type_Neuro ICU,icu_type_MICU,icu_type_SICU,icu_type_Cardiac ICU,icu_type_CSICU,apache_2_diagnosis_113.0,apache_2_diagnosis_108.0,apache_2_diagnosis_122.0,apache_2_diagnosis_203.0,apache_2_diagnosis_119.0,apache_2_diagnosis_301.0,apache_2_diagnosis_116.0,apache_2_diagnosis_112.0,apache_2_diagnosis_303.0,apache_2_diagnosis_218.0,apache_2_diagnosis_304.0,apache_2_diagnosis_302.0,apache_2_diagnosis_305.0,apache_2_diagnosis_124.0,apache_2_diagnosis_202.0,apache_2_diagnosis_207.0,apache_2_diagnosis_110.0,apache_2_diagnosis_209.0,apache_2_diagnosis_109.0,apache_2_diagnosis_106.0,apache_2_diagnosis_117.0,apache_2_diagnosis_120.0,apache_2_diagnosis_nan,apache_2_diagnosis_217.0,apache_2_diagnosis_114.0,apache_2_diagnosis_102.0,apache_2_diagnosis_308.0,apache_2_diagnosis_105.0,apache_2_diagnosis_212.0,apache_2_diagnosis_219.0,apache_2_diagnosis_306.0,apache_2_diagnosis_121.0,apache_2_diagnosis_214.0,apache_2_diagnosis_123.0,apache_2_diagnosis_213.0,apache_2_diagnosis_208.0,apache_2_diagnosis_101.0,apache_2_diagnosis_118.0,apache_2_diagnosis_307.0,apache_2_diagnosis_215.0,apache_2_diagnosis_103.0,apache_2_diagnosis_115.0,apache_2_diagnosis_104.0,apache_2_diagnosis_216.0,apache_2_diagnosis_107.0,apache_3j_diagnosis_502.01,apache_3j_diagnosis_203.01,apache_3j_diagnosis_703.03,apache_3j_diagnosis_1206.03,apache_3j_diagnosis_601.01,apache_3j_diagnosis_403.01,apache_3j_diagnosis_501.05,apache_3j_diagnosis_103.01,apache_3j_diagnosis_107.01,apache_3j_diagnosis_1304.08,apache_3j_diagnosis_1505.02,apache_3j_diagnosis_1406.01,apache_3j_diagnosis_1304.05,apache_3j_diagnosis_109.14,apache_3j_diagnosis_410.01,apache_3j_diagnosis_1506.03,apache_3j_diagnosis_1703.05,apache_3j_diagnosis_501.06,apache_3j_diagnosis_305.01,apache_3j_diagnosis_1204.01,apache_3j_diagnosis_1602.12,apache_3j_diagnosis_104.01,apache_3j_diagnosis_1302.02,apache_3j_diagnosis_305.02,apache_3j_diagnosis_108.01,apache_3j_diagnosis_212.01,apache_3j_diagnosis_601.04,apache_3j_diagnosis_106.01,apache_3j_diagnosis_407.01,apache_3j_diagnosis_0.25,apache_3j_diagnosis_1205.01,apache_3j_diagnosis_1502.02,apache_3j_diagnosis_212.02,apache_3j_diagnosis_102.01,apache_3j_diagnosis_206.01,apache_3j_diagnosis_306.01,apache_3j_diagnosis_1207.01,apache_3j_diagnosis_1204.02,apache_3j_diagnosis_207.01,apache_3j_diagnosis_111.01,apache_3j_diagnosis_501.04,apache_3j_diagnosis_1403.02,apache_3j_diagnosis_1504.03,apache_3j_diagnosis_405.01,apache_3j_diagnosis_1203.02,apache_3j_diagnosis_802.04,apache_3j_diagnosis_109.08,apache_3j_diagnosis_1602.06,apache_3j_diagnosis_208.01,apache_3j_diagnosis_402.02,apache_3j_diagnosis_211.09,apache_3j_diagnosis_1902.04,apache_3j_diagnosis_309.01,apache_3j_diagnosis_109.07,apache_3j_diagnosis_703.01,apache_3j_diagnosis_110.01,apache_3j_diagnosis_1506.07,apache_3j_diagnosis_409.02,apache_3j_diagnosis_211.03,apache_3j_diagnosis_1401.01,apache_3j_diagnosis_702.01,apache_3j_diagnosis_109.02,apache_3j_diagnosis_703.02,apache_3j_diagnosis_1405.01,apache_3j_diagnosis_1

In [180]:
corr = df_encoded.corr().abs()

In [183]:
s = corr.unstack()
so = s.sort_values(ascending = False, kind="quiksort")
type(so)

pandas.core.series.Series

In [185]:
filtered_so = so < 0

In [186]:
filtered_so

diabetes_mellitus            diabetes_mellitus              False
apache_3j_diagnosis_1409.02  apache_3j_diagnosis_1409.02    False
apache_3j_diagnosis_1705.02  apache_3j_diagnosis_1705.02    False
apache_3j_diagnosis_602.19   apache_3j_diagnosis_602.19     False
apache_3j_diagnosis_1802.02  apache_3j_diagnosis_1802.02    False
apache_3j_diagnosis_604.04   apache_3j_diagnosis_604.04     False
apache_3j_diagnosis_1502.01  apache_3j_diagnosis_1502.01    False
apache_3j_diagnosis_1604.03  apache_3j_diagnosis_1604.03    False
apache_3j_diagnosis_1304.06  apache_3j_diagnosis_1304.06    False
apache_3j_diagnosis_1408.13  apache_3j_diagnosis_1408.13    False
apache_3j_diagnosis_1602.01  apache_3j_diagnosis_1602.01    False
apache_3j_diagnosis_0.13     apache_3j_diagnosis_0.13       False
apache_3j_diagnosis_704.06   apache_3j_diagnosis_704.06     False
apache_3j_diagnosis_1302.01  apache_3j_diagnosis_1302.01    False
apache_3j_diagnosis_701.01   apache_3j_diagnosis_701.01     False
apache_3j_

# Notes

Are there areas where we may want to disaggregate the data? ex. it is likely that lab ranges and risk factors different for men vs. women?

look at attribute correlation